In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from bs4 import BeautifulSoup
import requests

In [2]:
#cargamos los 3 datasets que vamos a utlizzar para crear las funciones
dfjuegos=pd.read_csv('Juegos_steam.csv')
dfreviews=pd.read_csv('revies_por_usuario.csv')
dfitems=pd.read_csv('items_por_usuario.csv')

In [3]:
"""def PlayTimeGenre( genero : str ): #Debe devolver año con mas horas jugadas para dicho género.
Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}"""

'def PlayTimeGenre( genero : str ): #Debe devolver año con mas horas jugadas para dicho género.\nEjemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}'

In [4]:
#se verifica que no haya nulos, en caso que haya alguno filtrado
print(f'verificando años nulos:{dfjuegos.isnull().sum()}')

verificando años nulos:Id_juego           0
Nombre             0
Desarrollador      0
Año_lanzamiento    0
Género             0
dtype: int64


In [5]:
#verificamos que no haya valores duplicados
print(dfjuegos.duplicated().sum())
#quitamos valores duplicados
dfjuegos.drop_duplicates(inplace=True)

3


In [6]:
#quitamos los corchetes y comillas a la columna de generos
dfjuegos.Género=dfjuegos.Género.str.replace("[\[\]\']",'',regex=True)

In [7]:
#nos quedamos con las columnas que si vamos a usar para la función
dfjuegos=dfjuegos[['Id_juego','Nombre','Año_lanzamiento','Género',]]
dfjuegos.head()

,Id_juego,Nombre,Año_lanzamiento,Género
0,761140,Lost Summoner Kitty,2018,"Action, Casual, Indie, Simulation, Strategy"
1,643980,Ironbound,2018,"Free to Play, Indie, RPG, Strategy"
2,670290,Real Pool 3D - Poolians,2017,"Casual, Free to Play, Indie, Simulation, Sports"
3,767400,弹炸人2222,2017,"Action, Adventure, Casual"
4,773570,Log Challenge,2018,"Action, Casual, Indie, Sports"


In [8]:
# primero sumamos las horas de cada juego en total
itemsf1=dfitems[['Id_juego','Horas_jugadas']]
dff1=itemsf1.groupby('Id_juego').sum().reset_index()
dff1

,Id_juego,Horas_jugadas
0,10,285131.65
1,20,16009.26
2,30,12606.34
3,40,2573.98
4,50,12109.32
...,...,...
10973,528580,0.00
10974,528660,3.03
10975,529670,0.00
10976,529820,0.00


In [9]:
#rellenamos el dataframe dff1 con los años de lanzamiento y el genero
dff1=dff1.merge(dfjuegos,how='left')
dff1.head()

,Id_juego,Horas_jugadas,Nombre,Año_lanzamiento,Género
0,10,285131.65,Counter-Strike,2000,Action
1,20,16009.26,Team Fortress Classic,1999,Action
2,30,12606.34,Day of Defeat,2003,Action
3,40,2573.98,Deathmatch Classic,2001,Action
4,50,12109.32,Half-Life: Opposing Force,1999,Action


In [10]:
dff1

,Id_juego,Horas_jugadas,Nombre,Año_lanzamiento,Género
0,10,285131.65,Counter-Strike,2000,Action
1,20,16009.26,Team Fortress Classic,1999,Action
2,30,12606.34,Day of Defeat,2003,Action
3,40,2573.98,Deathmatch Classic,2001,Action
4,50,12109.32,Half-Life: Opposing Force,1999,Action
...,...,...,...,...,...
10973,528580,0.00,Found,2016,"Adventure, Casual, Free to Play, Indie, Early ..."
10974,528660,3.03,Spin Rush,2016,"Action, Casual, Indie"
10975,529670,0.00,NaN,NaN,NaN
10976,529820,0.00,Beyond Power VR,2016,"Action, Indie"


In [11]:
#tenemos datos faltantes que no existen en el dataframe dfjuegos, se procede priemro a hcaer webscrapping para obtener el dato
print(dff1.isnull().sum())
sindato=0
restantes=len(dff1.Id_juego[dff1.Género.isnull()])
#haciendo webscrapping, con los datos que tenemos podemos sacar el genero faltante para algunos juegos, ya que no todos nos arroja el resutlado de la pagina
for a in dff1.Id_juego[dff1.Género.isnull()].unique():
    url='https://store.steampowered.com/app/'+str(a)
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    genero =  soup.find('span', {'data-panel': '{"flow-children":"row"}'})
    año = soup.select_one('#game_highlights > div.rightcol > div > div.glance_ctn_responsive_left > div.release_date > div.date')
    if genero==None:
        sindato+=1
        restantes-=1
        continue
    elif año==None:
        continue
    else:
        año=año.text.split(',')[-1]
        genero = genero.a.text
        restantes-=1 
        dff1.loc[dff1.Id_juego==a,'Género']=genero
        dff1.loc[dff1.Id_juego==a,'Año_lanzamiento']=año
print('ids sin dato:',sindato)


Id_juego              0
Horas_jugadas         0
Nombre             1786
Año_lanzamiento    1786
Género             1786
dtype: int64
ids sin dato: 1253


In [12]:
# se rellenas datos nulos y los lugares donde es igual a 'N/D' con la frase 'sin dato'
dff1.loc[:,'Año_lanzamiento'].fillna('Sin dato',inplace=True) 
dff1.loc[:,'Género'].fillna('Sin dato',inplace=True)
dff1.loc[dff1.Año_lanzamiento=='N/D','Año_lanzamiento']='Sin dato'
dff1.loc[dff1.Género=='N/D','Género']='Sin dato'

In [13]:
#sacamos dummies de los generos
dummies=dff1.Género.str.get_dummies(', ')
dff1=pd.concat([dff1,dummies],axis=1)

In [14]:
dff1.head()

,Id_juego,Horas_jugadas,Nombre,Año_lanzamiento,Género,Accounting,Action,Adventure,Animation & Modeling,Audio Production,...,RPG,Racing,Simulation,Sin dato,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
0,10,285131.65,Counter-Strike,2000,Action,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20,16009.26,Team Fortress Classic,1999,Action,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30,12606.34,Day of Defeat,2003,Action,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,40,2573.98,Deathmatch Classic,2001,Action,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,50,12109.32,Half-Life: Opposing Force,1999,Action,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
#verificamos que no haya nulos ni duplicaods por ultima vez
print('nulos:',dff1.isnull().sum().sum())
print('duplicaods',dff1.duplicated().sum())

nulos: 1786
duplicaods 0


In [16]:
# se hace un Dataframe vacio para guardar las columnas filtradas
dff1_final=pd.DataFrame()

In [17]:
#obtenemos una lista de generos
generos=dummies.columns.to_list()
generos

['Accounting',
 'Action',
 'Adventure',
 'Animation & Modeling',
 'Audio Production',
 'Casual',
 'Design & Illustration',
 'Early Access',
 'Education',
 'Free to Play',
 'Game Development',
 'Indie',
 'Massively Multiplayer',
 'Photo Editing',
 'RPG',
 'Racing',
 'Simulation',
 'Sin dato',
 'Software Training',
 'Sports',
 'Strategy',
 'Utilities',
 'Video Production',
 'Web Publishing']

In [18]:
#se agrupa por año y genero, sumando las horas por genero en cada año
for a in generos:
    if dff1[dff1[a]==1].groupby(by=['Año_lanzamiento']).sum()['Horas_jugadas'].array.size==0:
        continue
    else:
        dftemp1=dff1[dff1[a]==1].groupby(by=['Año_lanzamiento']).sum()['Horas_jugadas'].reset_index()
        gen=[a for b in range(len(dftemp1))]
        dftemp1['Género']=pd.Series(gen)
        dftemp1=dftemp1.sort_values(by='Horas_jugadas',ascending=False)
        dftemp1=dftemp1.iloc[[0]]
        dff1_final=pd.concat([dff1_final,dftemp1])
 
dff1_final

,Año_lanzamiento,Horas_jugadas,Género
0,2016,0.00,Accounting
47,2012,18095017.31,Action
42,2011,3696751.15,Adventure
6,2015,22432.61,Animation & Modeling
6,2014,7591.02,Audio Production
39,2015,1399187.09,Casual
2,2012,32172.58,Design & Illustration
1,2013,1981714.67,Early Access
4,2013,5676.36,Education
16,2013,3153102.28,Free to Play


In [19]:
# guardamos en un csv el datafrme listo para la funcion
dff1_final.to_csv('Función_PlayTimeGenre.csv',index=False)

In [20]:
def PlayTimeGenre(genero):
    dict={}
    genero=genero.capitalize()
    mensaje="Año de lanzamiento con más horas jugadas para Género "+genero
    dict[mensaje]=dff1_final.Año_lanzamiento[dff1_final['Género']==genero].iloc[0]
    return dict

In [21]:
PlayTimeGenre('action')

{'Año de lanzamiento con más horas jugadas para Género Action': '2012'}

In [22]:

"""def UserForGenre( genero : str ): #Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.
Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}"""

'def UserForGenre( genero : str ): #Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.\nEjemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}'

In [23]:
dfitems.head()

,Id_juego,Nombre_juego,Horas_jugadas,Id_usuario
0,10,Counter-Strike,0.10,76561197970982479
1,20,Team Fortress Classic,0.00,76561197970982479
2,30,Day of Defeat,0.12,76561197970982479
3,40,Deathmatch Classic,0.00,76561197970982479
4,50,Half-Life: Opposing Force,0.00,76561197970982479


In [24]:
# primero traemos las columnas necesarias y quitaamos las filas donde las horas de jeugo sea 0
itemsf2=dfitems[['Id_usuario','Id_juego','Horas_jugadas']]
itemsf2.head()


,Id_usuario,Id_juego,Horas_jugadas
0,76561197970982479,10,0.10
1,76561197970982479,20,0.00
2,76561197970982479,30,0.12
3,76561197970982479,40,0.00
4,76561197970982479,50,0.00


In [25]:
itemsf2

,Id_usuario,Id_juego,Horas_jugadas
0,76561197970982479,10,0.10
1,76561197970982479,20,0.00
2,76561197970982479,30,0.12
3,76561197970982479,40,0.00
4,76561197970982479,50,0.00
...,...,...,...
5094087,76561198329548331,346330,0.00
5094088,76561198329548331,373330,0.00
5094089,76561198329548331,388490,0.05
5094090,76561198329548331,521570,0.07


In [26]:
#rectificamos que no haya nulos ni duplicados
print('nulos:',itemsf2.isnull().sum().sum())
print('duplicados: ',itemsf2.duplicated().sum())

nulos: 0
duplicados:  0


In [27]:
# de nuestra primera funcion traemos los años de lanzamiento y generos faltantes ya que ya se obtuvieron esos datos previamente
dfaños_generos=dff1[['Id_juego', 'Año_lanzamiento', 'Género', 'Accounting',
       'Action', 'Adventure', 'Animation & Modeling', 'Audio Production',
       'Casual', 'Design & Illustration', 'Early Access', 'Education',
       'Free to Play', 'Game Development', 'Indie', 'Massively Multiplayer',
       'Photo Editing', 'RPG', 'Racing', 'Simulation', 'Sin dato',
       'Software Training', 'Sports', 'Strategy', 'Utilities',
       'Video Production', 'Web Publishing']]
itemsf2=itemsf2.merge(dfaños_generos,how='left')
itemsf2

,Id_usuario,Id_juego,Horas_jugadas,Año_lanzamiento,Género,Accounting,Action,Adventure,Animation & Modeling,Audio Production,...,RPG,Racing,Simulation,Sin dato,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
0,76561197970982479,10,0.10,2000,Action,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,76561197970982479,20,0.00,1999,Action,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,76561197970982479,30,0.12,2003,Action,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,76561197970982479,40,0.00,2001,Action,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,76561197970982479,50,0.00,1999,Action,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5094087,76561198329548331,346330,0.00,2016,"Action, Adventure, Free to Play, Indie, Early ...",0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5094088,76561198329548331,373330,0.00,2015,"Adventure, Free to Play",0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5094089,76561198329548331,388490,0.05,2015,"Adventure, Free to Play",0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5094090,76561198329548331,521570,0.07,2016,"Casual, Free to Play, Indie",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
#verificamos que todos los campos esten rellenos
itemsf2.isnull().sum().sum()

0

In [29]:
#verificamos los tipos de dato de cada columna
itemsf2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5094092 entries, 0 to 5094091
Data columns (total 29 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Id_usuario             object 
 1   Id_juego               int64  
 2   Horas_jugadas          float64
 3   Año_lanzamiento        object 
 4   Género                 object 
 5   Accounting             int64  
 6   Action                 int64  
 7   Adventure              int64  
 8   Animation & Modeling   int64  
 9   Audio Production       int64  
 10  Casual                 int64  
 11  Design & Illustration  int64  
 12  Early Access           int64  
 13  Education              int64  
 14  Free to Play           int64  
 15  Game Development       int64  
 16  Indie                  int64  
 17  Massively Multiplayer  int64  
 18  Photo Editing          int64  
 19  RPG                    int64  
 20  Racing                 int64  
 21  Simulation             int64  
 22  Sin dato          

In [30]:
#se cambia la columan de años para poder agrupar
itemsf2['Año_lanzamiento']=itemsf2.Año_lanzamiento.astype(str)

In [31]:
# se crea un dataframe vacio para guardar las columnas agrupadas
dff2=pd.DataFrame()
#se agrupa por año y genero, sumando las horas por genero en cada año
for genero in generos:
       usuario=itemsf2.loc[itemsf2[genero]==1,['Id_usuario','Horas_jugadas']].groupby('Id_usuario').sum().sort_values(by='Horas_jugadas',ascending=False).iloc[0].name
       dftemp1=itemsf2.loc[(itemsf2.Id_usuario==usuario),['Id_usuario','Año_lanzamiento','Horas_jugadas']]
       dftemp1[['Género']]=genero
       dff2=pd.concat([dff2,dftemp1])
dff2

,Id_usuario,Año_lanzamiento,Horas_jugadas,Género
161428,76561198042410388,2000,0.10,Accounting
161429,76561198042410388,1999,8.58,Accounting
161430,76561198042410388,2003,0.02,Accounting
161431,76561198042410388,2001,0.03,Accounting
161432,76561198042410388,1999,0.28,Accounting
...,...,...,...,...
632728,Xyphien,2016,0.00,Web Publishing
632729,Xyphien,2016,3.92,Web Publishing
632730,Xyphien,2019,0.08,Web Publishing
632731,Xyphien,Sin dato,0.00,Web Publishing


In [32]:
dff2

,Id_usuario,Año_lanzamiento,Horas_jugadas,Género
161428,76561198042410388,2000,0.10,Accounting
161429,76561198042410388,1999,8.58,Accounting
161430,76561198042410388,2003,0.02,Accounting
161431,76561198042410388,2001,0.03,Accounting
161432,76561198042410388,1999,0.28,Accounting
...,...,...,...,...
632728,Xyphien,2016,0.00,Web Publishing
632729,Xyphien,2016,3.92,Web Publishing
632730,Xyphien,2019,0.08,Web Publishing
632731,Xyphien,Sin dato,0.00,Web Publishing


In [33]:
dff2=pd.DataFrame()
#se agrupa por año y genero, sumando las horas por genero en cada año
for a in generos:
    if itemsf2[itemsf2[a]==1].groupby(by=['Id_usuario','Año_lanzamiento']).sum()['Horas_jugadas'].array.size==0:
        continue
    else:
        dftemp1=pd.DataFrame(itemsf2[itemsf2[a]==1].groupby(by=['Id_usuario','Año_lanzamiento']).sum()['Horas_jugadas']).reset_index()
        dftemp2=pd.DataFrame(itemsf2[itemsf2[a]==1].groupby(by=['Id_usuario']).sum()['Horas_jugadas']).reset_index()
        usuario=dftemp2.sort_values(by='Horas_jugadas',ascending=False).iloc[0,0]
        genero=[a for b in range(len(dftemp1[dftemp1.Id_usuario==usuario]))]
        dftemp3=dftemp1[dftemp1.Id_usuario==usuario].copy()
        dftemp3.loc[:,'Género']=genero
        

        dff2=pd.concat([dff2,dftemp3])

In [34]:
dff2

,Id_usuario,Año_lanzamiento,Horas_jugadas,Género
0,76561198042410388,2016,0.00,Accounting
579641,Sp3ctre,2004,0.13,Action
579642,Sp3ctre,2009,2627.47,Action
579643,Sp3ctre,2010,1433.10,Action
579644,Sp3ctre,2011,58.61,Action
...,...,...,...,...
2102,76561198073642113,2014,3460.85,Utilities
2681,ScottyG555,2015,2805.23,Video Production
2381,Xyphien,2005,121.60,Web Publishing
2382,Xyphien,2012,1077.62,Web Publishing


In [35]:
#guardamos en un csv
dff2.to_csv('Función_UserForGenre.csv',index=False)

In [36]:
#creamos la función pedida en la consigna
def UserForGenre(genero):
    genero=genero.capitalize()
    l1=[]
    for a in dff2.loc[dff2.Género==genero,'Año_lanzamiento']:
        if dff2.loc[(dff2.Año_lanzamiento==a)&(dff2.Género==genero),'Horas_jugadas'].iloc[0]==0:
            continue
        else:
            horas=dff2.loc[(dff2.Año_lanzamiento==a)&(dff2.Género==genero),'Horas_jugadas'].iloc[0]
            l1.append({'Año':a,'Horas':horas})
    usuario=dff2.Id_usuario[dff2.Género==genero].unique()[0]

    return {f"Usuario con más horas jugadas para Género {genero}:": usuario, 'Horas jugadas':l1}

    

In [37]:
print(UserForGenre('action'))

{'Usuario con más horas jugadas para Género Action:': 'Sp3ctre', 'Horas jugadas': [{'Año': ' 2004', 'Horas': 0.13}, {'Año': ' 2009', 'Horas': 2627.47}, {'Año': ' 2010', 'Horas': 1433.1}, {'Año': ' 2011', 'Horas': 58.61}, {'Año': ' 2012', 'Horas': 1503.07}, {'Año': ' 2013', 'Horas': 4.12}, {'Año': ' 2014', 'Horas': 1983.38}, {'Año': ' 2020', 'Horas': 686.68}, {'Año': '1995', 'Horas': 3.62}, {'Año': '1999', 'Horas': 0.73}, {'Año': '2000', 'Horas': 1177.4}, {'Año': '2001', 'Horas': 0.22}, {'Año': '2002', 'Horas': 3.97}, {'Año': '2003', 'Horas': 127.88}, {'Año': '2004', 'Horas': 2123.51}, {'Año': '2005', 'Horas': 355.65}, {'Año': '2006', 'Horas': 14.940000000000001}, {'Año': '2007', 'Horas': 1879.74}, {'Año': '2008', 'Horas': 3.73}, {'Año': '2009', 'Horas': 1805.4400000000003}, {'Año': '2010', 'Horas': 1301.39}, {'Año': '2011', 'Horas': 2581.6}, {'Año': '2012', 'Horas': 6304.96}, {'Año': '2013', 'Horas': 3379.1000000000004}, {'Año': '2014', 'Horas': 2178.16}, {'Año': '2015', 'Horas': 5219.

In [38]:
#def UsersRecommend( año : int ): #Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
#Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [39]:
dfreviews.head()

,Id_usuario,Id_juego,Recommend,Año_review,sentiment_analysis
0,76561197970982479,1250,True,2011-01-01,2
1,js41637,22200,True,2011-01-01,2
2,evcentric,43110,True,2011-01-01,1
3,doctr,251610,True,2014-01-01,2
4,maplemage,227300,True,2013-01-01,1


In [40]:
dfreviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25771 entries, 0 to 25770
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Id_usuario          25771 non-null  object
 1   Id_juego            25771 non-null  int64 
 2   Recommend           25771 non-null  bool  
 3   Año_review          25771 non-null  object
 4   sentiment_analysis  25771 non-null  int64 
dtypes: bool(1), int64(2), object(2)
memory usage: 830.6+ KB


In [41]:
#cambiamos a formato fecha la columna año
dfreviews['Año_review']=pd.to_datetime(dfreviews.Año_review)

In [42]:
#filtramos los datos, donde Recommend sea igual a True y seniment analysis sea igual a 1 y 2
dfreviewsf3=dfreviews.loc[(dfreviews.Recommend==True)&(dfreviews.sentiment_analysis==2)|(dfreviews.Recommend==True)&(dfreviews.sentiment_analysis==1),['Id_juego','Año_review']].copy()

In [43]:
#sacmos una lista de años, y despues vamos iterando por cada añopara localizar el top 3 de los juegos mas recomendados para cada año
años=dfreviews.Año_review.dt.year.sort_values().unique().tolist()
dff3=pd.DataFrame()
for a in años:
   dftemp1=dfreviewsf3.loc[(dfreviews.Año_review.dt.year==a)].value_counts(ascending=False).iloc[:3].to_frame().reset_index().copy()
   dff3=pd.concat([dff3,dftemp1])
#guaradamos en un Dataframe el top 3 por cada año
dff3=dff3.reset_index(drop=True)
dff3

,Id_juego,Año_review,count
0,440,2010-01-01,5
1,630,2010-01-01,4
2,1250,2010-01-01,3
3,440,2011-01-01,33
4,620,2011-01-01,13
5,105600,2011-01-01,6
6,440,2012-01-01,106
7,105600,2012-01-01,29
8,4000,2012-01-01,17
9,440,2013-01-01,249


In [44]:
#traemos el nombre de cada juego uniendo con su id de juego
dff3=pd.merge(dfjuegos[['Id_juego','Nombre']],dff3).sort_values(by=['Año_review','count'],ascending=False).drop(columns='Id_juego')
dff3

,Nombre,Año_review,count
9,Counter-Strike: Global Offensive,2016-01-01,321
20,Team Fortress 2,2016-01-01,134
4,Garry's Mod,2016-01-01,79
8,Counter-Strike: Global Offensive,2015-01-01,633
19,Team Fortress 2,2015-01-01,251
3,Garry's Mod,2015-01-01,151
18,Team Fortress 2,2014-01-01,514
7,Counter-Strike: Global Offensive,2014-01-01,356
2,Garry's Mod,2014-01-01,278
17,Team Fortress 2,2013-01-01,249


In [45]:
#convertimos la columna de años, a modo que solo quede el año numerico
dff3.Año_review=dff3.Año_review.dt.year

In [46]:
#guardamos en un csv
dff3.to_csv('Función_UsersRecommend.csv',index=False)

In [47]:
#creamos la función pedida en la consigna
def UsersRecommend( año ):
    l=[]
    for a in range(3):
        dict={f'Puesto {a+1}':dff3[dff3.Año_review==año].iloc[a,0]}
        l.append(dict)
    return l
    
    #Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [71]:
print(UsersRecommend(2014))

[{'Puesto 1': 'Team Fortress 2'}, {'Puesto 2': 'Counter-Strike: Global Offensive'}, {'Puesto 3': "Garry's Mod"}]


In [49]:
#def UsersWorstDeveloper( año : int ): #Devuelve el top 3 de desarrolladoras con juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)
#Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [50]:
#filtramos los datos donde Reccommend seas igual a False y sentiment analysis sea igual a 0
dfreviewsf4=dfreviews.loc[(dfreviews.Recommend==False)&(dfreviews.sentiment_analysis==0),['Id_juego','Año_review']].copy()
dfreviewsf4.head()

,Id_juego,Año_review
130,344760,2015-01-01
396,400800,2016-01-01
524,215470,2013-01-01
535,226700,2014-01-01
686,224540,2016-01-01


In [51]:
dfjuegos2=pd.read_csv('Juegos_steam.csv')
dfjuegos2=dfjuegos2[['Id_juego','Desarrollador']]
dfjuegos2.head()

,Id_juego,Desarrollador
0,761140,Kotoshiro
1,643980,Secret Level SRL
2,670290,Poolians.com
3,767400,彼岸领域
4,773570,N/D


In [52]:
#traemos el nombre del desarrollador
dff4=dfreviewsf4.merge(dfjuegos2[['Id_juego','Desarrollador']],how='left')
print(len(dff4))
dff4.head()


293


,Id_juego,Año_review,Desarrollador
0,344760,2015-01-01,Code}{atch
1,400800,2016-01-01,Milestone S.r.l.
2,215470,2013-01-01,Lukewarm Media
3,226700,2014-01-01,NaN
4,224540,2016-01-01,Jagex Limited


In [53]:
#haciendo webscrapping, con los datos que tenemos podemos sacar el genero faltante oara algunos juegos, ya que no todos nos arroja el resutlado de la pagina
for a in dff4.Id_juego[dff4.Desarrollador.isnull()].unique():
    url='https://store.steampowered.com/app/'+str(a)
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    elemento_div = soup.find('div', {'class': 'summary column', 'id': 'developers_list'})
    if elemento_div==None:
        continue
    else:
        texto_enlace = elemento_div.a.text
     
    dff4.loc[dff4.Id_juego==a,'Desarrollador']=texto_enlace

In [54]:
#ahora donde tenemos datos 'N/D' hacemos el mismo proceso
for a in dff4.Id_juego[dff4.Desarrollador=='N/D'].unique():
    url='https://store.steampowered.com/app/'+str(a)
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    elemento_div = soup.find('div', {'class': 'summary column', 'id': 'developers_list'})
    if elemento_div==None:
        continue
    else:
        texto_enlace = elemento_div.a.text
    dff4.loc[dff4.Id_juego==a,'Desarrollador']=texto_enlace

In [55]:
# despues comprobamos los valores que tenemos restantes
len(dff4[(dff4['Desarrollador'].isnull())|(dff4['Desarrollador']=='N/D')])


25

In [56]:
#dff4.dropna(inplace=True)

In [57]:
# quitamos la columna Id_juego que ya no utlizaremos
dff4.drop(columns='Id_juego',inplace=True)

In [58]:
# se realiza un Dataframe con los datos necesarios para la función
dff4_final=pd.DataFrame()
for a in dff4.Año_review.unique():
    dff4_final=pd.concat([dff4_final,dff4[dff4.Año_review==a].value_counts(ascending=False).reset_index()[:3]])


In [59]:
dff4_final

,Año_review,Desarrollador,count
0,2015-01-01,Facepunch Studios,4
1,2015-01-01,Valve,4
2,2015-01-01,Bohemia Interactive,4
0,2016-01-01,Valve,11
1,2016-01-01,Hello Games,8
2,2016-01-01,Bohemia Interactive,4
0,2013-01-01,Capcom,1
1,2013-01-01,City Interactive,1
2,2013-01-01,Eyedentity Games Inc.,1
0,2014-01-01,Infinity Ward,4


In [60]:
#sacamos el año en forma numérica
dff4_final['Año_review']=dff4_final.Año_review.dt.year

In [61]:
#guardamos en una funcion los datos para nuestra funcion
dff4_final.to_csv('Función_UsersWorstDeveloper.csv',index=False)

In [62]:
#creamos la función pedida en la consigna
def UsersWorstDeveloper( año ):
    dict={}
    for a,b in enumerate(dff4_final.loc[dff4_final.Año_review==año,'Desarrollador']):
        dict['Puesto '+str(a+1)]=b
    return [dict]


In [72]:
UsersWorstDeveloper(2012)

[{'Puesto 1': 'Valve'}]

In [ ]:
#def sentiment_analysis( empresa desarrolladora : str ): #Según la empresa desarrolladora, se devuelve un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor.
#Ejemplo de retorno: {'Valve' : [Negative = 182, Neutral = 120, Positive = 278]}

In [ ]:
#seleccionamos las columnas de reviews que usaremos
dfreviewsf5=dfreviews[['Id_juego','sentiment_analysis']].copy()
dfreviewsf5.head()

,Id_juego,sentiment_analysis
0,1250,2
1,22200,2
2,43110,1
3,251610,2
4,227300,1


In [ ]:
#agregamos las columnas de Desarrollador del dataframe de juegos
dfreviewsf5=pd.merge(dfjuegos2[['Id_juego','Desarrollador']],dfreviewsf5,how='right')
dfreviewsf5.head()

,Id_juego,Desarrollador,sentiment_analysis
0,1250,Tripwire Interactive,2
1,22200,ACE Team,2
2,43110,NaN,1
3,251610,NaN,2
4,227300,SCS Software,1


In [ ]:
#hacemos un dataframe 
dff5=pd.get_dummies(dfreviewsf5,columns=['sentiment_analysis'],dtype=int).drop(columns='Id_juego').groupby('Desarrollador').count().reset_index()

In [ ]:
#cambiamos el formato de todos los desarrolladores, a modo que todos empiecen con letra mayúscula y el resto sea minúscula
dff5['Desarrollador']=dff5['Desarrollador'].str.capitalize()

In [ ]:
# la columna Desarrollador tiene valores que no deberian estar, como (Mac), lo cual afecta a los resultados de la agurapción 
#se hace una lista donde la palabra mac y linux se en cuentre presente, y se quita de la candea de strings
l=[]
for a in dff5.Desarrollador:
    b=0
    c=0    
    if '(' in a:
        b=a.find('(')
    if ')' in a:
        c=a.find(')')
        if 'mac' in a[b:c+1] and  a[b:c+1] not in l:
            l.append(a[b:c+1])
        if 'linux' in a[b:c+1] and a[b:c+1] not in l:
            l.append(a[b:c+1])
for a in l:
    dff5.loc[dff5.Desarrollador.str.contains(a,regex=False),'Desarrollador']=dff5.loc[dff5.Desarrollador.str.contains(a,regex=False),'Desarrollador'].str.replace(a,'')

In [ ]:
#comprobamos que no se haya quedado algún valor pendiente de cambiar
l=[]
for a in dff5.Desarrollador:
    b=0
    c=0    
    if '(' in a:
        b=a.find('(')
    if ')' in a:
        c=a.find(')')
        if 'mac' in a[b:c+1] and  a[b:c+1] not in l:
            l.append(a[b:c+1])
        if 'linux' in a[b:c+1] and a[b:c+1] not in l:
            l.append(a[b:c+1])
l

[]

In [ ]:
#cambiamos manualmente los campos donde aparece ubisoft, por ser un desarrollador importante
dff5.loc[dff5.Desarrollador.str.contains('Ubisoft'),'Desarrollador']='Ubisoft'
dff5.loc[dff5.Desarrollador.str.contains(' in collaboration with ubisoft reflections'),'Desarrollador']=dff5.loc[dff5.Desarrollador.str.contains(' in collaboration with ubisoft reflections'),'Desarrollador'].str.replace(' in collaboration with ubisoft reflections','ubisoft')
dff5.loc[dff5.Desarrollador.str.contains('ubisoft montreal'),'Desarrollador']=dff5.loc[dff5.Desarrollador.str.contains('ubisoft montreal'),'Desarrollador'].str.replace('ubisoft montreal','ubisoft')
dff5.loc[dff5.Desarrollador.str.contains('ubisoft  shanghaï'),'Desarrollador']=dff5.loc[dff5.Desarrollador.str.contains('ubisoft  shanghaï'),'Desarrollador'].str.replace('ubisoft  shanghaï','ubisoft')
dff5.loc[dff5.Desarrollador.str.contains('ubisoft kiev'),'Desarrollador']=dff5.loc[dff5.Desarrollador.str.contains('ubisoft kiev'),'Desarrollador'].str.replace('ubisoft kiev','ubisoft')

In [ ]:
#hacemos una copia del dataframe
dff5_prueba=dff5.copy()

In [ ]:
#separamos los desarrolladores, ya que en algunos casos tienen mas de dos desarrolladores,
dff5_prueba.loc[:,'Desarrollador']=dff5.Desarrollador.str.split(',')

dff5_prueba=dff5_prueba.explode('Desarrollador')

In [ ]:
#se cambia todo de nuevo a formato de primer letra mayúscula, y el resto minúsculas
dff5_prueba.Desarrollador=dff5_prueba.Desarrollador.str.strip('. ').str.capitalize()
dff5_prueba.Desarrollador=dff5_prueba.Desarrollador.str.strip('. ').str.capitalize()

In [ ]:
#en la empresa square enix, normalizamso todo para que solo aparezca Square enix
dff5_prueba.loc[dff5_prueba.Desarrollador.str.contains('quare'),'Desarrollador']='Square enix'

#en la lista de valores que su cadena es de largo hasta 3, quitamos los valores que si corresponden a desarrolladoras
l=dff5_prueba.Desarrollador[dff5_prueba.Desarrollador.str.len()<4].unique().tolist()
print(l)
l.remove('橘子班')
l.remove('インレ')
l.remove('Kog')
l.remove('M2h')


['Agc', 'Llc', 'Inc', 'Ltd', 'Ccp', 'Kog', 'M2h', 'N/d', 'Sos', 'S.l', 'インレ', '橘子班']


In [ ]:
#ahora quitamos del daaframe los valores que no son de desarrolladoras,
for a in l:
    dff5_prueba.drop(index=dff5_prueba.loc[dff5_prueba.Desarrollador==a].index,inplace=True)
#comprobamos que solo quedan los valores correspondientes a los que si son empresas desrrolladoras
dff5_prueba.Desarrollador[dff5_prueba.Desarrollador.str.len()<4].unique()

array(['Kog', 'M2h', 'インレ', '橘子班'], dtype=object)

In [ ]:
#agrupadmos por las desarrolladoras, sumando los valores de cada columan del sentyment analisis para cada valor de 0,1 y 2
dff5_prueba=dff5_prueba.groupby(by='Desarrollador').sum().reset_index()


,Año_review,Desarrollador
0,2015-01-01,Code}{atch
1,2016-01-01,Milestone S.r.l.
2,2013-01-01,Lukewarm Media
3,2014-01-01,Fredaikis AB
4,2016-01-01,Jagex Limited


In [ ]:
#checamos como quedó la lista de empresas desarrolladoras
for a in dff5_prueba.Desarrollador:
    print(a)

07th expansion
10tons ltd
11 bit studios
16bit nights
17-bit
1c-avalon
1c-softclub
1c: maddox games
1c:inoco
22cans
2d boy
2k australia
2k boston
2k china
2k czech
2k marin
2play
3 sprockets
30.06 studios ltd
3000ad
343 industries
38 studios
3909
3f interactive
4 i lab
48h studio
4a games
5 bits games
5 lives studios
5th cell media
5wolf
777 studios
8-bit studio
800 north and digital ranch
8888888 labs
9heads game studios
@unepic_fran
A barnett
A jolly corpse
Abbey games
Above and beyond technologies
Abraham carreola
Abrakam sa
Abstract tree studio
Abstraction games
Access games
Ace maddox
Ace team
Acid nerve
Acquire corp
Actalogic
Adam foster
Adrian zingg
Adventure planning service
Afterthought llc
Airtight games
Akakiy petrushkin
Alan edwardes
Alawar
Alawar entertainment
Albert fernandez
Aleksey abramenko
Alexander bruce
Alice &amp; smith
Alice in dissonance
Alien pixel studios
Alientrap
Allen dayan
Alligator pit
Allplayer group ltd
Almost human games
Alpixel games
Altar games
Amanit

In [ ]:
#guardamos en un csv los datos para la función
dff5_prueba.to_csv('Función_sentiment_analysis.csv', index=False)

In [ ]:
def sentiment_analysis( empresa_desarrolladora ): 
#Según la empresa desarrolladora, se devuelve un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor.
#Ejemplo de retorno: {'Valve' : [Negative = 182, Neutral = 120, Positive = 278]}
   Negative=dff5.sentiment_analysis_0[dff5.Desarrollador==empresa_desarrolladora].iloc[0]
   Neutral=dff5.sentiment_analysis_1[dff5.Desarrollador==empresa_desarrolladora].iloc[0]
   Positive = dff5.sentiment_analysis_2[dff5.Desarrollador==empresa_desarrolladora].iloc[0]
   l= f'[Negative = {Negative}, Neutral =  {Neutral}, Positive = {Positive}]'
   return {empresa_desarrolladora:l}

In [ ]:
sentiment_analysis('Valve')

{'Valve': '[Negative = 3816, Neutral =  3816, Positive = 3816]'}